In [1]:
import pandas as pd
import re
import matplotlib.pyplot as plt

Load, Clean and tokenise the text

In [2]:
train_df = pd.read_csv('train_2kmZucJ.csv')
X_train = train_df['tweet']
Y_train = train_df['label']
train_df.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...


In [3]:
import re

def cleaner(text):
    # Remove URLs
    text = re.sub(r'https?://\S+', ' ', text)
    # Remove special characters that aren't punctuation
    text = re.sub(r"[^a-zA-Z0-9.,!?\s]", " ", text)
    text = text.lower()
    tokens = text.split()
    return " ".join(tokens)
train_df['Clean Text'] = train_df['tweet'].apply(cleaner)

In [4]:
train_df['Clean Text'][1]

'finally a transparant silicon case thanks to my uncle yay sony xperia s sonyexperias'

Tokenisation of Text


In [5]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer

X_train,X_test,Y_train,Y_test = train_test_split(train_df['Clean Text'],train_df['label'],test_size=0.2,random_state=42,shuffle=True)
max_value_length = len(str(X_train.max()))
X_tokenizer = Tokenizer()
X_tokenizer.fit_on_texts(X_train)
X_tokenizer.word_index

{'iphone': 1,
 'apple': 2,
 'i': 3,
 'my': 4,
 'the': 5,
 'to': 6,
 'samsung': 7,
 'a': 8,
 'and': 9,
 'it': 10,
 'new': 11,
 'twitter': 12,
 'for': 13,
 'com': 14,
 'you': 15,
 'me': 16,
 'phone': 17,
 'sony': 18,
 'is': 19,
 'follow': 20,
 'of': 21,
 'on': 22,
 'in': 23,
 'this': 24,
 's': 25,
 't': 26,
 'with': 27,
 'pic': 28,
 'ipad': 29,
 'so': 30,
 'love': 31,
 'like': 32,
 'life': 33,
 'at': 34,
 'just': 35,
 'rt': 36,
 'have': 37,
 'android': 38,
 'all': 39,
 'now': 40,
 'day': 41,
 'that': 42,
 'your': 43,
 'an': 44,
 'instagram': 45,
 'can': 46,
 'gain': 47,
 'today': 48,
 'cute': 49,
 'not': 50,
 'photography': 51,
 'photo': 52,
 'm': 53,
 'back': 54,
 'get': 55,
 'from': 56,
 'news': 57,
 'fun': 58,
 'be': 59,
 'case': 60,
 'got': 61,
 'galaxy': 62,
 'music': 63,
 'app': 64,
 'instagood': 65,
 'out': 66,
 'ios': 67,
 'who': 68,
 'but': 69,
 'happy': 70,
 'beautiful': 71,
 'lol': 72,
 'are': 73,
 'fashion': 74,
 'funny': 75,
 'time': 76,
 'smile': 77,
 'birthday': 78,
 'no':

In [6]:
len(X_tokenizer.word_index)
threshold = 3
cnt = 0
for key,value in X_tokenizer.word_index.items():
    if value >= threshold:
        cnt+=1
print(cnt)
X_tokenizer = Tokenizer(num_words=cnt,oov_token='unk')
X_tokenizer.fit_on_texts(X_train)

15794


In [20]:
from keras.preprocessing.sequence import pad_sequences
#79 is the longest length sequence so set the padding to 100
max_len = 100
X_train_seq = X_tokenizer.texts_to_sequences(X_train)
X_test_seq = X_tokenizer.texts_to_sequences(X_test)
X_train_seq = pad_sequences(X_train_seq,padding='post',maxlen = max_len)
X_test_seq = pad_sequences(X_test_seq,padding='post',maxlen = max_len)

In [21]:
x_voc_size = X_tokenizer.num_words

In [22]:
print(len(X_train_seq[0]))

100


In [23]:
from keras.models import *
from keras.layers import *
from keras.callbacks import *
model = Sequential()
model.add(Embedding(x_voc_size,50,input_shape=(max_len,),mask_zero=True))
#rnn
model.add(SimpleRNN(128,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

c:\Users\Srikant Santosh\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [24]:
model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 100, 50)        │       789,700 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_2 (SimpleRNN)        │ (None, 128)            │        22,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 829,253 (3.16 MB)

 Trainable params: 829,253 (3.16 MB)

 Non-trainable params: 0 (0.00 B)

In [25]:
model.compile(optimizer='adam',loss='binary_crossentropy')


In [26]:
mc = ModelCheckpoint("weights.best.keras",monitor='val_loss',verbose=1,save_best_only=True,mode='min')

Model Training 10 epochs of batch size 128


In [27]:
model.fit(X_train_seq,Y_train,batch_size=128,epochs=10,verbose=1,validation_data=(X_test_seq,Y_test),callbacks=[mc])


Epoch 1/10
49/50 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.5734
Epoch 1: val_loss improved from inf to 0.34585, saving model to weights.best.keras
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - loss: 0.5691 - val_loss: 0.3458
Epoch 2/10
49/50 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.3075
Epoch 2: val_loss improved from 0.34585 to 0.28448, saving model to weights.best.keras
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - loss: 0.3065 - val_loss: 0.2845
Epoch 3/10
49/50 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.1773
Epoch 3: val_loss improved from 0.28448 to 0.26695, saving model to weights.best.keras
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - loss: 0.1770 - val_loss: 0.2669
Epoch 4/10
49/50 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.0983
Epoch 4: val_loss did not improve from 0.26695
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - loss: 0.0980 - val_loss: 0.3540
Epoch 5/10
49/50 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.0399
Epoch 5: val_loss did not improve from 0.26695
50/50 ━━━━━━━━━━━━━━━━━

In [34]:
from sklearn import metrics
model.load_weights("weights.best.keras")
results_predicted = model.predict(X_test_seq)
predicted_class = (model.predict(X_test_seq) > 0.5).astype("int32")

Y_pred = pd.Series(predicted_class.flatten(), name="Predicted_Class")
print(metrics.classification_report(Y_test,Y_pred))

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
              precision    recall  f1-score   support

           0       0.94      0.89      0.92      1152
           1       0.75      0.84      0.79       432

    accuracy                           0.88      1584
   macro avg       0.84      0.87      0.85      1584
weighted avg       0.89      0.88      0.88      1584



In [43]:
test_file = r'test_12QyDcx.csv'
df = pd.read_csv(test_file)
X = df.tweet
X_clean = X.apply(cleaner)
X_clean_seq = X_tokenizer.texts_to_sequences(X_clean)
X = pad_sequences(X_clean_seq,padding='post',maxlen = max_len)
results_example_file = (model.predict(X) > 0.5).astype('int')
df['Label'] = results_example_file
df.to_csv('Submission.csv')


62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
